In [ ]:
# Starting with last weeks lab scripts

In [ ]:
#  The following bash scripts download the elastic search library and install it
# on the google colab instance.
# Credit for the bash scripts and server run start goes to:
# https://gist.github.com/korakot/15fe4f18d0e0f53d7b834ef797880500

#  You need to run these only once when you work on your search engine notebook

# NOTE: If you are working on a large dataset (20k+ docs) you should do this locally
# i.e. in a jupyter notebook. This way you only need to install ES once and
# index your data ones

In [ ]:
!wget -q https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-oss-7.9.2-linux-x86_64.tar.gz
!wget -q https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-oss-7.9.2-linux-x86_64.tar.gz.sha512
!tar -xzf elasticsearch-oss-7.9.2-linux-x86_64.tar.gz
!sudo chown -R daemon:daemon elasticsearch-7.9.2/
!shasum -a 512 -c elasticsearch-oss-7.9.2-linux-x86_64.tar.gz.sha512

elasticsearch-oss-7.9.2-linux-x86_64.tar.gz: OK


In [ ]:
# # https://stackoverflow.com/questions/68762774/elasticsearchunsupportedproducterror-the-client-noticed-that-the-server-is-no#answer-68918449
!pip install elasticsearch==7.9.1 -q

In [ ]:
# check elasticsearch version in environment
!pip freeze | grep elasticsearch

elasticsearch==7.9.1


In [ ]:
# import utility packages
import urllib.request
from bs4 import BeautifulSoup
import re
import time
import random

# let's import ES
from elasticsearch import Elasticsearch

In [ ]:
%%bash --bg
sudo -H -u daemon elasticsearch-7.9.2/bin/elasticsearch

Starting job # 0 in a separate thread.


In [ ]:
%%bash
ps -ef | grep elasticsearch

root         197       1  0 08:20 ?        00:00:00 sudo -H -u daemon elasticsearch-7.9.2/bin/elasticsearch
daemon       198     197  6 08:20 ?        00:00:21 /content/elasticsearch-7.9.2/jdk/bin/java -Xshare:auto -Des.networkaddress.cache.ttl=60 -Des.networkaddress.cache.negative.ttl=10 -XX:+AlwaysPreTouch -Xss1m -Djava.awt.headless=true -Dfile.encoding=UTF-8 -Djna.nosys=true -XX:-OmitStackTraceInFastThrow -XX:+ShowCodeDetailsInExceptionMessages -Dio.netty.noUnsafe=true -Dio.netty.noKeySetOptimization=true -Dio.netty.recycler.maxCapacityPerThread=0 -Dio.netty.allocator.numDirectArenas=0 -Dlog4j.shutdownHookEnabled=false -Dlog4j2.disable.jmx=true -Djava.locale.providers=SPI,COMPAT -Xms1g -Xmx1g -XX:+UseG1GC -XX:G1ReservePercent=25 -XX:InitiatingHeapOccupancyPercent=30 -Djava.io.tmpdir=/tmp/elasticsearch-9890396109560329245 -XX:+HeapDumpOnOutOfMemoryError -XX:HeapDumpPath=data -XX:ErrorFile=logs/hs_err_pid%p.log -Xlog:gc*,gc+age=trace,safepoint:file=logs/gc.log:utctime,pid,tags:filecou

In [ ]:
# start es server
time.sleep(20) # give the server 20 seconds to start ..
!curl -X GET "http://localhost:9200"

{
  "name" : "63e8d0868ea8",
  "cluster_name" : "elasticsearch",
  "cluster_uuid" : "gRm00zHZRsSxxCxVk-uNBg",
  "version" : {
    "number" : "7.9.2",
    "build_flavor" : "oss",
    "build_type" : "tar",
    "build_hash" : "d34da0ea4a966c4e49417f2da2f244e3e97b4e6e",
    "build_date" : "2020-09-23T00:45:33.626720Z",
    "build_snapshot" : false,
    "lucene_version" : "8.6.2",
    "minimum_wire_compatibility_version" : "6.8.0",
    "minimum_index_compatibility_version" : "6.0.0-beta1"
  },
  "tagline" : "You Know, for Search"
}


In [ ]:
def test_ES(es):
  """
  Script to test whether we have succesfully started an ES instance and
  and imported the python library
  """

  return es.ping()  # got True


In [ ]:
# start and TEST es
es = Elasticsearch("http://localhost:9200")
if test_ES(es):
  print('ES instance working')
else:
  print('ES instance not working')

ES instance working


In [ ]:
# Server information
#es.info()

### Data frame

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import csv

df = pd.read_csv('/content/gdrive/MyDrive/netflix_titles.csv')
df

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...
...,...,...,...,...,...,...,...,...,...,...,...,...
8802,s8803,Movie,Zodiac,David Fincher,"Mark Ruffalo, Jake Gyllenhaal, Robert Downey J...",United States,"November 20, 2019",2007,R,158 min,"Cult Movies, Dramas, Thrillers","A political cartoonist, a crime reporter and a..."
8803,s8804,TV Show,Zombie Dumb,NaN,NaN,NaN,"July 1, 2019",2018,TV-Y7,2 Seasons,"Kids' TV, Korean TV Shows, TV Comedies","While living alone in a spooky town, a young g..."
8804,s8805,Movie,Zombieland,Ruben Fleischer,"Jesse Eisenberg, Woody Harrelson, Emma Stone, ...",United States,"November 1, 2019",2009,R,88 min,"Comedies, Horror Movies",Looking to survive in a world taken over by zo...
8805,s8806,Movie,Zoom,Peter Hewitt,"Tim Allen, Courteney Cox, Chevy Chase, Kate Ma...",United States,"January 11, 2020",2006,PG,88 min,"Children & Family Movies, Comedies","Dragged from civilian life, a former superhero..."


In [ ]:
#drop NaN values
df = df.dropna()

In [ ]:
#Remove special-character letters from the description column
description_edited = df['description'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
print(description_edited)

7       On a photo shoot in Ghana, an American model s...
8       A talented batch of amateur bakers face off in...
9       A woman adjusting to life after a loss contend...
12      After most of her family is murdered in a terr...
24      When the father of the man she loves insists t...
                              ...                        
8801    Recovering alcoholic Talal wakes up inside a s...
8802    A political cartoonist, a crime reporter and a...
8804    Looking to survive in a world taken over by zo...
8805    Dragged from civilian life, a former superhero...
8806    A scrappy but poor boy worms his way into a ty...
Name: description, Length: 5332, dtype: object


In [ ]:
#Substitute description by description_edited
df2 = description_edited.to_frame()
df = df.append(df2)

#drop NaN values
df = df.dropna()
df

#Convert into release year into integer type rather than float

df['release_year'] = df['release_year'].fillna(0).astype(int)

In [ ]:
# df

In [ ]:
# mappings are used to define what kind of structure your data has. here explicit mapping is used:
# https://www.elastic.co/guide/en/elasticsearch/reference/current/explicit-mapping.html

# The mapping is used when creating the index through the request body:

request_body = {
    'settings': {
        'number_of_shards': 1,
        'number_of_replicas': 1,

    },
    'mappings': {
          'properties': {
              # 'show_id': {'type': 'text'},
              'title': {'type': 'text'},
              'description': {'type': 'text'},
              'listed_in': {'type': 'text'},
              'release_year': {'type': 'integer'}

          }
    }
}

index_name = 'test'
try:
    es.indices.get(index_name)
    print('index {} already exists'.format(index_name))
except:
    print('creating index {}'.format(index_name))
    es.indices.create(index_name, body=request_body)

creating index test


In [ ]:
# # now what we want to do is put some data in the index, i.e. index it:
# for show_id, title, description, listed_in, release_year in df:
#     doc_body = {
#       'show_id': show_id,
#       'title': title,
#       'description': description,
#       'listed_in': listed_in,
#       'release_year': release_year
#   }
#     es.index(index_name, doc_body)

In [ ]:
# # now what we want to do is put some data in the index, i.e. index it:
for df.show_id, df.title, df.description, df.listed_in, df.release_year in df:
    doc_body = {
      'show_id': df.show_id,
      'title': df.title,
      'description': df.description,
      'listed_in': df.listed_in,
      'release_year': df.release_year
  }
    es.index(index_name, doc_body)

ValueError: ignored

In [ ]:
def index_info(index_name):
  count, deleted, shards, =  es.cat.indices(index=index_name, h=['docs.count', 'docs.deleted', 'pri'])[:-1].split(' ')
  print(
      """
      #### INDEX INFO #####
      index_name = {}
      doc_count = {}
      shard_count = {}
      deleted_doc_count = {}
      """.format(index_name, count, shards, deleted)
  )

In [ ]:
index_info(index_name)

In [ ]:
def search(index_name, query_body):
  # return only doc_id and rank
  results = es.search(index=index_name, body=query_body, explain=False)
  plain_results = [(x['_source']['title'], x['_source']['release_year'], x['_score']) for x in results['hits']['hits']]
  return results, plain_results

In [ ]:
query_body = {
    'query':{
        'term': {
            'body':  'ship'
        }
    }
}
results, plain_results = search(index_name, query_body)

In [ ]:
for title, release_year, score in plain_results:
    print(title, release_year, score)

### UI

# Simple UI with Colab Forms

Click the `⋮` button on the cell to hide/show code.


In [ ]:
#@title Default title text
query = 'a'  #@param {type:"string"}
#before_year = 1987 #@param {type:"number"}
#after_year = 1984 #@param {type:"number"}
before_year_slider = 1980 #@param {type:"slider", min:1980, max:2010, step:1}
after_year_slider = 2010 #@param {type:"slider", min:1980, max:2010, step:1}

query_terms = query.split(' ')


# term_search = [{
#                     "term": {
#                         "body": t
#                     }
#                 } for t in query_terms]

query_body = {
    "query":{
        "bool":{
            "should": [
                {
                    "term": {
                        "body": query
                    }
                },
                {
                    "range":{
                        "year": {
                            "gte": after_year_slider,
                            "lte": before_year_slider
                        }

                    }
                }
            ]
        }
    }
}
# query_body['query']['bool']['should'].append(term_search)
results, plain_results = search(index_name, query_body)
for title, release_year, score in plain_results:
    print(title, release_year, score)

## ElasticSearch API Resources

* [Creating an Index](https://www.elastic.co/guide/en/elasticsearch/reference/current/indices-create-index.html)
* [Using IR models](https://www.elastic.co/guide/en/elasticsearch/reference/current/index-modules-similarity.html)
* [Mappings](https://www.elastic.co/guide/en/elasticsearch/reference/current/mapping.html)
* [Shards](https://opster.com/guides/elasticsearch/glossary/elasticsearch-shards/)
